In [1]:
from model import *
from torch.optim import AdamW,Adam
from torch.nn.utils import clip_grad_value_

In [2]:
transform = A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=12, p=1)
data_train = ImgDataset('omniglot-py/images_background',n,k,64,transform)
data_train = DataLoader(data_train,b)
data_val = ImgDataset('omniglot-py/images_evaluation',n,k,16,transform)
data_val = DataLoader(data_val,b)

In [3]:
lr = 1e-3
clip = 1e-2
epochs = 20

In [4]:
model = AttenMeta(d, dropout,layers,True).to('cuda')
paras = model.parameters()
opt = AdamW(paras,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)

In [5]:

opt.zero_grad()
for epoch in range(epochs):

    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    # Training #
    for i,x in enumerate(data_train):
        x = x.to('cuda:0')
        loss = model(x,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()
        
    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(data_val):
            x = x.to('cuda:0')
            loss = model(x,True)
            val_loss += loss.item()      
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))

epoch:0, train_loss:2.20591422460847, val_loss:1.9901869032118056
epoch:1, train_loss:2.4035414639165844, val_loss:1.9908437463972304
epoch:2, train_loss:1.9636973728567868, val_loss:1.9908438788519964
epoch:3, train_loss:3.1592164463916066, val_loss:3.002240127987332
epoch:4, train_loss:2.0346201193534723, val_loss:1.930519209967719
epoch:5, train_loss:1.8811687554343273, val_loss:1.9914298587375217
epoch:6, train_loss:1.874765004141856, val_loss:1.9827213552263048
epoch:7, train_loss:1.875710143881329, val_loss:1.9845434029897053
epoch:8, train_loss:1.8668965666981068, val_loss:1.989876561694675
epoch:9, train_loss:1.8689891706078738, val_loss:1.9950204425387912
epoch:10, train_loss:1.8592079029244892, val_loss:1.9965544011857774
epoch:11, train_loss:1.8620023323317705, val_loss:1.9833628336588542
epoch:12, train_loss:1.8591048818523601, val_loss:1.9900559054480658
epoch:13, train_loss:1.8590969150349246, val_loss:1.9908437463972304
epoch:14, train_loss:1.854126457440651, val_loss:1.

In [6]:
acc = 0
model.eval()
with torch.no_grad():
    for j,data in enumerate(data_val):
        x = x.to('cuda:0')
        yhat = model(x,False)
        acc += torch.sum(yhat.argmax(-1).flatten() == model.y)/b/n
print(acc/j)

tensor(0.2083, device='cuda:0')


In [11]:
1/6, -np.log(1/6)

(0.16666666666666666, 1.791759469228055)